In [29]:
import os

from sqlalchemy import create_engine, text
import pandas as pd
from dotenv import load_dotenv

load_dotenv("../.envrc")


True

In [30]:
METRICS_USER = os.environ["METRICS_USER"]
METRICS_PASSWORD = os.environ["METRICS_PASSWORD"]
METRICS_HOSTNAME = os.environ["METRICS_HOSTNAME"]
METRICS_PORT = os.environ["METRICS_PORT"]
METRICS_DB = os.environ["METRICS_DB"]

In [31]:
engine = create_engine(
    f"postgresql://{METRICS_USER}:{METRICS_PASSWORD}@{METRICS_HOSTNAME}:{METRICS_PORT}/{METRICS_DB}"
)

def default_metrics_extraction(query: str ):
    with engine.connect() as connection:
        df_raw = pd.read_sql_query(text(query), connection)
    return df_raw

In [32]:
base_query = """
                SELECT
                    ov0.value AS "value",
                    ov0.unit AS "unit",
                    fhirql_code(occ0.code) AS "code",
                    lower(o0."effectiveDateTime") AS "observation_date",
                    o0.id AS "obs_id",
                    p1.id AS "patient_id"
                FROM
                    observation o0
                    JOIN "observation_valueQuantity" ov0 ON ov0._resource = o0._id
                    JOIN observation_code oc0 ON oc0._resource = o0._id
                    JOIN observation_code_coding occ0 ON occ0._resource = o0._id
                    JOIN observation_subject os0 ON os0._resource = o0._id
                    JOIN patient p1 ON p1._id = os0._reference_id AND os0._reference_type = 'Patient'
                WHERE
                    fhirql_code(occ0.code) IN ({codes})
                LIMIT 5
            """

# MCV_fL
dieselben Einheiten

In [33]:
df = default_metrics_extraction(query=base_query.format(codes="'zMCV'"))
df

,value,unit,code,observation_date,obs_id,patient_id
0,83.3,fl,zMCV,2022-07-19 06:29:00+00:00,241a339d6ee08040bfb5b1c803c4c4e9dc5ae15d6185eb...,9d8581039f90c456c31af5634f2b5c223b508f18e94f97...
1,84.4,fl,zMCV,2022-07-15 09:27:00+00:00,24250b26f2d5ed172e2b9d843f8165f3f5131fd590bf04...,85e370e9d77606c5c7f0052a89fe0d392261a3262a3f3b...
2,84.2,fl,zMCV,2022-07-20 07:41:00+00:00,b0a13b7b58b4e24b8ad44818b26ff228dc594fe199eb0f...,85e370e9d77606c5c7f0052a89fe0d392261a3262a3f3b...
3,84.5,fl,zMCV,2022-07-19 07:34:00+00:00,ca511543fed5ba6dfcf70ca40a9cc1ae4fb81d3f18394c...,85e370e9d77606c5c7f0052a89fe0d392261a3262a3f3b...
4,83.7,fl,zMCV,2022-06-15 08:43:00+00:00,96e3e36dcf0dabc3be50732cff20ecf64a620af59ca073...,cefdc60d402d9d0008b564103de3f10b1c90eeb8866e7d...


# PT_percent
Bei diesem Wert bin ich noch etwas unsicher, weil die Bezeichnung nicht Typisch für Deutschland ist. Trotz Recherche bleibe ich unsicher. Aber ich glaube der swisslab wert muss einfach durch 10 geteilt werden

In [34]:
df = default_metrics_extraction(query=base_query.format(codes="'zTPZCS'"))
df

,value,unit,code,observation_date,obs_id,patient_id
0,120.0002,%,zTPZCS,2022-07-15 09:27:00+00:00,badfbf8e62519049f054c3002c6a5b8f3213b9062cc1c4...,85e370e9d77606c5c7f0052a89fe0d392261a3262a3f3b...
1,102.0000,%,zTPZCS,2022-07-14 10:39:00+00:00,8b0a5b2a3c5c06ef0ca67878e3c8fa411969a8079fcb32...,965383b7b3c39144f70b7b479ef6537a4c6a4052c64c46...
2,67.0000,%,zTPZCS,2022-07-15 01:45:00+00:00,21416e2b947f3b31dd98e2fdf27b4c8d4e5addd538645f...,965383b7b3c39144f70b7b479ef6537a4c6a4052c64c46...
3,86.0000,%,zTPZCS,2022-07-19 09:41:00+00:00,3fb2eb26755ff8ba08ad27c4ae90e241d204d254ee3a6e...,1edb39c317246e8c549dc0b906dd845aa0842dcef8f8a4...
4,85.0000,%,zTPZCS,2022-07-19 17:45:00+00:00,524b20a4db86d8bcbe2d4e3a6c066fde1edbcfeac08478...,1edb39c317246e8c549dc0b906dd845aa0842dcef8f8a4...


# LDH_UI_L
dieselben Einheiten

In [35]:
df = default_metrics_extraction(query=base_query.format(codes="'zLDH'"))
df

,value,unit,code,observation_date,obs_id,patient_id
0,258.0,U/l,zLDH,2022-06-15 08:43:00+00:00,1f0f50ebe7e0d01c8ea482bebe592dc2b407fcdb863c1d...,cefdc60d402d9d0008b564103de3f10b1c90eeb8866e7d...
1,219.0,U/l,zLDH,2022-07-19 09:34:00+00:00,cacf9606fc4a3baf3154434b198f4a5a48f5b37b2cbc63...,768be0c33b0254d68ba223728b819eb58d79ca0d63e962...
2,205.0,U/l,zLDH,2022-07-19 13:38:00+00:00,d23779319ab8533f9e4c33b168ed97514e42dfc4f834c6...,1103005311e240340fd10abfc57384ac4f47ba312dc113...
3,255.0,U/l,zLDH,2022-07-19 13:37:00+00:00,63a65b1f22f4c8eac1bbdb070618d22b24f16dbb85a8f4...,b94385ff8f1241db709f30a56f0b80f11a780190addda7...
4,437.0,U/l,zLDH,2022-07-14 10:39:00+00:00,d978f8be9a9c1cde2d3c2543c8b73e05ffbac14377e1bc...,965383b7b3c39144f70b7b479ef6537a4c6a4052c64c46...


# MCHC_g_L
zMCHC (mit Faktor 10 Multiprizieren vom UK Labor in die APP)

In [36]:
df = default_metrics_extraction(query=base_query.format(codes="'zMCHC'"))
df

,value,unit,code,observation_date,obs_id,patient_id
0,33.1,g/dl,zMCHC,2022-07-19 06:29:00+00:00,7575e8d93e22c9e338b323ad2ad4630e9a7d5170d214fc...,9d8581039f90c456c31af5634f2b5c223b508f18e94f97...
1,32.1,g/dl,zMCHC,2022-07-15 09:27:00+00:00,52ab883f018c2a751d63d63e53e66e6f229cba1e72790c...,85e370e9d77606c5c7f0052a89fe0d392261a3262a3f3b...
2,32.9,g/dl,zMCHC,2022-07-19 07:34:00+00:00,d63c6d627f414012f0f00536e15c7d36876d037bbc3c15...,85e370e9d77606c5c7f0052a89fe0d392261a3262a3f3b...
3,32.6,g/dl,zMCHC,2022-07-20 07:41:00+00:00,739eb1e3188eac9d1840e79101698f7025da914a024271...,85e370e9d77606c5c7f0052a89fe0d392261a3262a3f3b...
4,34.3,g/dl,zMCHC,2022-06-15 08:43:00+00:00,5d70b42df5545e6e3100642da912d6091fa533282dcf27...,cefdc60d402d9d0008b564103de3f10b1c90eeb8866e7d...


# WBC_G_L
alls Zellen/nl in swisslabg hinterlegt ist, kann der wert so übernommen werden

In [37]:
df = default_metrics_extraction(query=base_query.format(codes="'zLEUA'"))
df

,value,unit,code,observation_date,obs_id,patient_id
0,10.97,/nl,zLEUA,2022-07-19 06:29:00+00:00,dd5cf7907e937a4dbd12925c30dcb932620da517bbef24...,9d8581039f90c456c31af5634f2b5c223b508f18e94f97...
1,6.85,/nl,zLEUA,2022-07-15 09:27:00+00:00,21c0c7e46d1baebbfe1b1d90fac0635c38ae9326ca0c51...,85e370e9d77606c5c7f0052a89fe0d392261a3262a3f3b...
2,10.62,/nl,zLEUA,2022-07-19 07:34:00+00:00,144bfe8d23c78998bf5ac2d591f3f8bf9f27b209ceff30...,85e370e9d77606c5c7f0052a89fe0d392261a3262a3f3b...
3,9.45,/nl,zLEUA,2022-07-20 07:41:00+00:00,e0ea92df092f234219551a2ca2b052547d580b484ca2db...,85e370e9d77606c5c7f0052a89fe0d392261a3262a3f3b...
4,4.53,/nl,zLEUA,2022-06-15 08:43:00+00:00,7c483d22d658f0af31f949934a7e566e269da40a2dfaa7...,cefdc60d402d9d0008b564103de3f10b1c90eeb8866e7d...


# Fibrinogen_g_L
ich weiß nicht welche Einheiten im Swisslab liegen

In [38]:
df = default_metrics_extraction(query=base_query.format(codes="'zFIBRCS'"))
df

,value,unit,code,observation_date,obs_id,patient_id
0,286.0000,mg/dl,zFIBRCS,2022-07-14 10:39:00+00:00,725349d50c76486fee8d73649a625fb28310868e5b7992...,965383b7b3c39144f70b7b479ef6537a4c6a4052c64c46...
1,207.0000,mg/dl,zFIBRCS,2022-07-15 01:45:00+00:00,6e98aa3c12d3a5bdf947a2e08fc45f4668c008cfd5c544...,965383b7b3c39144f70b7b479ef6537a4c6a4052c64c46...
2,900.0002,mg/dl,zFIBRCS,2022-07-16 09:48:00+00:00,de7ad0ae92314194dac040ffc2f59e8af052abe24a6fb8...,8c7541839dd510c83e31f4d9749f908c4a00de7e0ae2df...
3,92.0000,mg/dl,zFIBRCS,2022-07-19 19:27:00+00:00,5aa29c7537a31786b4abbc0c5226d9d590d6d49f2a5ce7...,a3ada7b76306afe06448e0ef318381efe7c7ea12fdb4f2...
4,684.0000,mg/dl,zFIBRCS,2022-07-18 02:21:00+00:00,151022965652baf6a3b6fcc9f1d71bcca769bdc2a4e01b...,ddb5bede1dfc78a7fa7c86bca712905c23b96e9304932e...


# Monocytes_G_L
kann so übernommen werden, wenn ich mich nicht irre

In [39]:
df = default_metrics_extraction(query=base_query.format(codes="'zMON#'"))
df

,value,unit,code,observation_date,obs_id,patient_id
0,0.45,/nl,zMON#,2022-07-18 13:25:00+00:00,14b899ad09e4d93030510ad55bb3fb92f6bd6dad338934...,579c92012be95cfc2458eccd2d674c25888188ce023e5f...
1,0.45,/nl,zMON#,2022-07-19 13:37:00+00:00,cb967252dccaf0a6a1b82863ff887c948455130d6c321b...,3cc91d3015e262c7ad5f8d18cbdce0d3f1ca18ad65f0f8...
2,0.57,/nl,zMON#,2022-07-19 14:29:00+00:00,70b26f801d510c76714722b365b1514eac972c54adfad1...,72edb03e1665d84f2188ee97bff360575753bdaf2bb4dc...
3,1.13,/nl,zMON#,2022-07-20 07:42:00+00:00,8fff128be4973af102488020e561f4d7149c82089b1cfc...,b2d3591f881904195efe088f787db10175dbb7993263b7...
4,0.72,/nl,zMON#,2022-07-18 02:21:00+00:00,62a940345c74fef1a41c29d4537d98ef36bff692a4e65c...,ddb5bede1dfc78a7fa7c86bca712905c23b96e9304932e...


# Platelets_G_L
falls Zellen/nl in swisslabg hinterlegt ist, kann der wert so übernommen werden, sonst durch 1000 dividieren

In [40]:
df = default_metrics_extraction(query=base_query.format(codes="'zTHRA'"))
df

,value,unit,code,observation_date,obs_id,patient_id
0,309.0,/nl,zTHRA,2022-07-18 09:58:00+00:00,f5a36266846729e5b663349b8bc85dec253daab99f0344...,579c92012be95cfc2458eccd2d674c25888188ce023e5f...
1,327.0,/nl,zTHRA,2022-07-18 13:25:00+00:00,08a28b745da65ccc91aa1dd7d847ed8871a2be97b11e12...,579c92012be95cfc2458eccd2d674c25888188ce023e5f...
2,300.0,/nl,zTHRA,2022-07-19 07:40:00+00:00,fba6753c7d7de960c9bb5f8884f61eeb6aef9b35dd53e3...,579c92012be95cfc2458eccd2d674c25888188ce023e5f...
3,117.0,/nl,zTHRA,2022-07-13 09:22:00+00:00,dbc5371ed61b7701c2eadfecd8a267d6a76117896fee58...,cd5d117f05c0fcbf52ed22ff2eddef602da94adec03021...
4,157.0,/nl,zTHRA,2022-07-12 07:07:00+00:00,f3bc508d802632dd4703f142480f040df3260b6cbe9fe6...,cd5d117f05c0fcbf52ed22ff2eddef602da94adec03021...


# Lymphocytes_G_L
falls Zellen/nl in swisslabg hinterlegt ist, kann der wert so übernommen werden

In [41]:
df = default_metrics_extraction(query=base_query.format(codes="'zLYM#'"))
df

,value,unit,code,observation_date,obs_id,patient_id
0,1.76,/nl,zLYM#,2022-07-18 13:25:00+00:00,d85fc6f531d188ba93fc48a54a57af0864459dc72a4170...,579c92012be95cfc2458eccd2d674c25888188ce023e5f...
1,3.08,/nl,zLYM#,2022-07-19 13:37:00+00:00,d8e0ccc16eaeb91225f7b60237f711dbeff1b28c3ebef7...,3cc91d3015e262c7ad5f8d18cbdce0d3f1ca18ad65f0f8...
2,3.60,/nl,zLYM#,2022-07-19 14:29:00+00:00,8549ef8d24af4f15dc74a009d6318202519892f6c77e88...,72edb03e1665d84f2188ee97bff360575753bdaf2bb4dc...
3,3.03,/nl,zLYM#,2022-07-20 07:42:00+00:00,0d687cf4daacdfacbaa8de62aa50f46ce856249b8794c5...,b2d3591f881904195efe088f787db10175dbb7993263b7...
4,1.15,/nl,zLYM#,2022-07-18 02:21:00+00:00,4b1cbb8ebb0cc0d015bfeee2b631ebf3841cc7637616f7...,ddb5bede1dfc78a7fa7c86bca712905c23b96e9304932e...
